In [4]:
import urllib.request    
urllib.request.urlretrieve("https://www.actuarylist.com/", "test.txt")

('test.txt', <http.client.HTTPMessage at 0x1b333324f20>)

In [263]:
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup

def get_description(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        description_tag = soup.find("p",  text = "Job Description")       
        if description_tag:
            return description_tag.find_next('ul').text.strip()
        return "N/A"
    else:
        print(f"Failed to fetch this url Respones status is : {response.status_code} ")

def get_jobs(url):
    user_agents = [ 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
	'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
	'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
	'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
      ] 
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent} 
    response = requests.get(url , headers = headers)

    if response.status_code == 200:
        job_list = []
        soup = BeautifulSoup(response.text, "html.parser")
        job_cards = soup.find_all("article")
        for job in job_cards:
            job_title = job.find("p", class_="Job_job-card__position__ic1rc")
            job_salary = job.find("p", class_="Job_job-card__salary__QZswp")
            job_company = job.find("p" , class_="Job_job-card__company__7T9qY")
            job_country = job.find("a", class_="Job_job-card__country__GRVhK") 
            anchorText = job.find('a',class_="Job_job-page-link__a5I5g").get("href")
            #print(anchorText)
            #print("https://www.actuarylist.com" + anchorText)
            job_description = get_description("https://www.actuarylist.com" + anchorText)

            job_list.append({
                "Job Title": job_title.text.strip() if job_title else "N/A",
                "Company": job_company.text.strip() if job_company else "N/A",
                "Salary": job_salary.text.strip() if job_salary else "N/A",
                "Country" : job_country.text.strip() if job_country else "N/A",
                "Description": job_description
            })

        return job_list  

    else:
        print(f"Failed to fetch jobs, Status Code: {response.status_code}")
        return []

base_url = "https://www.actuarylist.com/" 
jobs = []
# pages = 570 / 30 = 19
for page in range(1, 20):
    url = base_url if page == 1 else f"{base_url}?page={page}"
    jobs.extend(get_jobs(url))

df = pd.DataFrame(jobs)
df.to_csv("actuarylist_jobs(Extracted).csv", index = False)
print(f"Saved {len(df)} extracted job listings to ActuaryList")

unique_df = df.drop_duplicates(subset = ['Job Title','Company','Country'],keep='first')
unique_df.to_csv("actuarylist_jobs(Cleaned).csv", index = False)
print(f"Saved {len(unique_df)} unique job listings to ActuaryList")


C:\Users\jamzh\AppData\Local\Temp\ipykernel_14568\4068944633.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description_tag = soup.find("p",  text = "Job Description")


Saved 558 extracted job listings to ActuaryList
Saved 462 unique job listings to ActuaryList
